In [1]:
pip install sentence-transformers gradio chromadb langchain langchain_core langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107

In [2]:
import json
from sentence_transformers import SentenceTransformer
import gradio as gr
import os
import chromadb
from chromadb.utils import embedding_functions
from langchain.chains import LLMChain
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
with open('/content/Alexander_Street_shareGPT_2.0.json', 'r') as file:
    dataset = json.load(file)
print("Dataset loaded successfully.")

Dataset loaded successfully.


In [4]:
chroma_client = chromadb.Client()
print("ChromaDB client initialized.")

# Define the embedding function
embedding_model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

# Define a custom embedding function
class CustomEmbeddingFunction:
    def __call__(self, input: str) -> list:
        # Use the embedding model to encode the input text
        embedding = embedding_model.encode(input)
        return embedding.tolist()

# Create an instance of the custom embedding function
embedding_func = CustomEmbeddingFunction()
print("Embedding function defined.")

# Create a collection in ChromaDB
collection = chroma_client.create_collection(
    name="mindguardian_collection",
    embedding_function=embedding_func,
    metadata={"hnsw:space": "cosine"}
)
print("ChromaDB collection created.")

from more_itertools import chunked

import uuid

def store_embedded_data_in_chromadb(dataset, batch_size=10):
    global_id = 0  # Start a global counter
    for batch in chunked(dataset, batch_size):
        ids = []
        documents = []
        metadatas = []

        for data in batch:
            merged_text = data['input'] + " " + data['output']
            ids.append(str(global_id))  # Use the global counter for unique IDs
            documents.append(merged_text)
            metadatas.append(data)  # Store original data as metadata
            global_id += 1  # Increment the global counter

        # Add data to the ChromaDB collection
        collection.add(
            ids=ids,
            documents=documents,
            metadatas=metadatas
        )
    print("Data stored in ChromaDB.")

ChromaDB client initialized.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Embedding function defined.
ChromaDB collection created.


In [5]:
store_embedded_data_in_chromadb(dataset)

Data stored in ChromaDB.


In [26]:
from langchain.llms import GooglePalm

token = "XXXXXXX"
llm_google_palm = GooglePalm(google_api_key=token, temperature=0.1, max_tokens= 100)

In [27]:
system_prompt = "<s>[INST] You are an expert mental health counseling chatbot named Mindguardian. You provide professional mental health counseling to users. [/INST]"
conversational_memory_length = 10
memory = ConversationBufferWindowMemory(k=conversational_memory_length, memory_key="chat_history", return_messages=True)
print("System prompt and memory initialized.")

System prompt and memory initialized.


In [28]:
def query_chromdb(user_query):
    # Generate the query vector from the user's input
    query_vector = embedding_model.encode(user_query).tolist()
    response = collection.query(
        query_embeddings=[query_vector],
        n_results=1,
        include=["metadatas"]  # Include embeddings in the response
    )
    return response

# Example usage
response = query_chromdb("I've been feeling a bit off. I sometimes find it hard to focus and concentrate on tasks, but it doesn't happen too often. I have been feeling a bit disconnected from my friends, but we still hang out and talk normally. My sleeping habits haven't changed much")
print(response['metadatas'])

[[{'input': "Lately, I've been feeling quite disconnected from the world. I sometimes find it hard to focus on my daily tasks and I've even started to withdraw from my friends and family. I don't know why, but I feel like they might be talking about me behind my back. The other day, I was at a party and it sounded like everyone was laughing at me. I do have mood swings as well, but they're not too frequent. My sleep has also been disturbed recently, but I don't see any unusual things or hear any weird voices.", 'instruction': 'If you are a counsellor, please answer the questions based on the description of the patient.', 'output': "Thank you for sharing your feelings with me. It's essential to understand that emotional distress can manifest in various ways, and you're not alone in this experience. It's important that you pay attention to how these symptoms impact your daily life. I would like to know more about the changes in your social interactions and the intensity of your mood swin

In [29]:

def query_llm(user_question, _):
    try:
        print(f"Received user question: {user_question}")
        context = query_chromdb(user_question)

        # Construct a chat prompt template using various components
        prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessage(
                    content=system_prompt
                ),
                MessagesPlaceholder(
                    variable_name="chat_history"
                ),
                SystemMessage(
                    content=f"<s>[INST] Use this context only if relevant to user query: {context} [/INST]"
                ),
                HumanMessagePromptTemplate.from_template(
                    "<s>[INST] User query: {human_input} [/INST]"
                ),
            ]
        )
        print("Prompt constructed.")

        # Create a conversation chain using the LangChain LLM
        conversation = LLMChain(
            llm=llm_google_palm,  # Use Mistral 7B model
            prompt=prompt,
            verbose=False,
            memory=memory,
        )
        print("LLMChain initialized.")
        response = conversation.predict(human_input=user_question)
        print(f"LLM response: {response}")

        return response
    except Exception as e:
        print(f"Error in query_llm: {e}")
        return "Sorry, something went wrong. Please try again."

# Default message for the chatbot
default_message = """I'm MindGuardian, a mental health counseling chatbot. How can I help you?"""

In [30]:
gradio_interface = gr.ChatInterface(
    query_llm,
    chatbot=gr.Chatbot(value=[[None, default_message]]),
    textbox=gr.Textbox(placeholder="Type your query", container=False, scale=7),
    title="Mindguardian, a mental health counseling chatbot",
    theme='gradio/base',
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
)

# Launch the interface
print("Launching Gradio interface...")
gradio_interface.launch(debug=True)

Launching Gradio interface...
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://38ae2767c207211c19.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Received user question: hi
Prompt constructed.
LLMChain initialized.
LLM response: Hello, how are you?
Received user question: not good actually. I was constantly bullied at school by saying I have a bent leg
Prompt constructed.
LLMChain initialized.
Error in query_llm: list index out of range
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://38ae2767c207211c19.gradio.live
